In [ ]:
# default_exp diffquant_utils

# Utility Functions general

In [ ]:
#export
def get_condpairname(condpair):
    return f"{condpair[0]}_VS_{condpair[1]}"

In [ ]:
#export
def get_middle_elem(sorted_list):
    nvals = len(sorted_list)
    if nvals==1:
        return sorted_list[0]
    middle_idx = nvals//2
    if nvals%2==1:
        return sorted_list[middle_idx]
    return 0.5* (sorted_list[middle_idx] + sorted_list[middle_idx-1])

In [ ]:
#export
import numpy as np
def get_nonna_array(array_w_nas):
    res = []
    isnan_arr = np.isnan(array_w_nas)

    for idx in range(len(array_w_nas)):
        sub_res = []
        sub_array = array_w_nas[idx]
        na_array = isnan_arr[idx]
        for idx2 in range(len(sub_array)):
            if not na_array[idx2]:
               sub_res.append(sub_array[idx2])
        res.append(np.array(sub_res))
    return np.array(res)

In [ ]:
#export
import numpy as np
def get_non_nas_from_pd_df(df):
    return {
        pep_name: sub_vals[~np.isnan(sub_vals)] for pep_name, sub_vals in
        zip( df.index.values, df.values)
    }

In [ ]:
#export
import numpy as np
def get_ionints_from_pd_df(df):
    return {
        pep_name: sub_vals for pep_name, sub_vals in
        zip( df.index.values, df.values)
    }

In [ ]:
#export
def invert_dictionary(my_map):
    inv_map = {}
    for k, v in my_map.iteritems():
        inv_map[v] = inv_map.get(v, []) + [k]
    return inv_map


In [ ]:
#export
from scipy.stats import norm

def get_z_from_p_empirical(p_emp,p2z):
    p_rounded = np.format_float_scientific(p_emp, 1)
    if p_rounded in p2z:
        return p2z.get(p_rounded)
    z = norm.ppf(float(p_rounded))
    p2z[p_rounded] = z
    return z

# Input Parsers
The Alphaquant pipeline is run using a generic wide-table input format, as specified in the documentation. The following parsers convert long format tables as provided e.g. by Spectronaut or DIA-NN into this generic format. The configuration for the parsers is set by a yaml file.

# Constants

In [ ]:
#export
ion_hierarchy = {"fragion" :["SEQ", "MOD", "CHARGE", "FRGION"], "ms1iso" : ["SEQ", "MOD", "CHARGE", "MS1ISOTOPES"]}

## Convert long format to wide format

### Parse .yaml file
The relevant parameters for reading and reformatting the long table are stored in the "intable_config.yaml" file. The functions below are for reading and reformating the config info

In [ ]:
#export
import yaml

def get_relevant_columns(protein_cols, ion_cols, sample_ID, quant_ID, filter_dict):
    filtcols = []
    for filtconf in filter_dict.values():
        filtcols.append(filtconf.get('param'))
    relevant_cols = protein_cols + ion_cols + [sample_ID] + [quant_ID] + filtcols
    relevant_cols = list(set(relevant_cols)) # to remove possible redudancies
    return relevant_cols


def get_relevant_columns_config_dict(config_typedict):
    filtcols = []
    dict_ioncols = []
    for filtconf in config_typedict.get('filters', {}).values():
        filtcols.append(filtconf.get('param'))

    for headr in config_typedict.get('ion_hierarchy', {}).values():
        dict_ioncols.extend(headr)
    quantID = config_typedict.get("quant_ID")
    if type(quantID) ==type("string"):
        quant_ids = [config_typedict.get("quant_ID")]
    elif quantID == None:
        quant_ids = []
    else:
        quant_ids = list(config_typedict.get("quant_ID").values())
    relevant_cols = config_typedict.get("protein_cols") + config_typedict.get("ion_cols", []) + [config_typedict.get("sample_ID")] + quant_ids + filtcols + dict_ioncols
    relevant_cols = list(set(relevant_cols)) # to remove possible redudancies
    return relevant_cols


def get_config_columns(config_dict):
    protein_cols = config_dict.get("protein_cols")
    ion_cols = config_dict.get("ion_cols")
    sample_ID = config_dict.get("sample_ID")
    quant_ID = config_dict.get("quant_ID")
    filter_dict = config_dict.get("filters", {})
    relevant_cols = get_relevant_columns(protein_cols, ion_cols, sample_ID, quant_ID, filter_dict)
    return relevant_cols, protein_cols, ion_cols, sample_ID, quant_ID, filter_dict


def load_config(config_yaml):
    stream = open(config_yaml, 'r')
    config_all = yaml.safe_load(stream)
    return config_all

def get_type2relevant_cols(config_all):
    type2relcols = {}
    for type in config_all.keys():
        config_typedict = config_all.get(type)
        relevant_cols = get_relevant_columns_config_dict(config_typedict)
        type2relcols[type] = relevant_cols
    return type2relcols


### Filter and reformat

In [ ]:
#export

def filter_input(filter_dict, input):
    for filtname,filterconf in filter_dict.items():
        param = filterconf.get('param')
        comparator = filterconf.get('comparator')
        value = filterconf.get('value')
        
        if comparator not in [">",">=", "<", "<=", "==", "!="]:
            raise TypeError(f"cannot identify the filter comparator of {filtname} given in the longtable config yaml!")

        if comparator=="==":
            input = input[input[param] ==value]
            continue
        try:
            input = input.astype({f"{param}" : "float"})
        except:
            pass

        if comparator==">":
            input = input[input[param].astype(type(value)) >value]

        if comparator==">=":
            input = input[input[param].astype(type(value)) >=value]

        if comparator=="<":
            input = input[input[param].astype(type(value)) <value]

        if comparator=="<=":
            input = input[input[param].astype(type(value)) <=value]
        
        if comparator=="!=":
            input = input[input[param].astype(type(value)) !=value]
        
    return input

In [ ]:
#export
def merge_protein_and_ion_cols(input_df, config_dict):
    protein_cols =  config_dict.get("protein_cols")
    ion_cols = config_dict.get("ion_cols")
    input_df['protein'] = input_df.loc[:, protein_cols].astype('string').sum(axis=1)
    input_df['ion'] = input_df.loc[:, ion_cols].astype('string').sum(axis=1)
    input_df = input_df.rename(columns = {config_dict.get('quant_ID') : "quant_val"})
    return input_df

In [ ]:
#export
def merge_protein_cols_and_ion_dict(input_df, config_dict):
    """[summary]

    Args:
        input_df ([pandas dataframe]): longtable containing peptide intensity data
        confid_dict ([dict[String[]]]): nested dict containing the parse information. derived from yaml file

    Returns:
        pandas dataframe: longtable with newly assigned "protein" and "ion" columns
    """
    protein_cols = config_dict.get("protein_cols")
    ion_dict = config_dict.get("ion_hierarchy")
    splitcol2sep = config_dict.get('split_cols')
    quant_id_dict = config_dict.get('quant_ID')

    ion_dfs = []
    input_df['protein'] = input_df.loc[:, protein_cols].astype('string').sum(axis=1)
    input_df = input_df
    for hierarchy_type in ion_hierarchy.keys():
        df_subset = input_df.copy()
        ion_hierarchy_local = [x for x in ion_hierarchy.get(hierarchy_type) if x in ion_dict]
        ion_headers_merged, ion_headers_grouped = get_relvant_column_names(ion_dict, ion_hierarchy_local) #ion headers merged is just a helper to select all relevant rows, ionheaders grouped contains the sets of ionstrings to be merged into a list eg [[SEQ, MOD], [CH]]
        
        if hierarchy_type == "ms1iso":
            headers = ion_headers_merged + [config_dict.get('sample_ID'), 'protein']
            df_subset = df_subset[headers].drop_duplicates()
            df_subset = df_subset.set_index(['protein', config_dict.get('sample_ID')])
            df_subset = split_extend_df(df_subset, splitcol2sep)
        else:
            headers = ion_headers_merged + ['protein', config_dict.get('sample_ID'), quant_id_dict.get(hierarchy_type)]
            df_subset = df_subset[headers].drop_duplicates()
            df_subset = df_subset.set_index(['protein', config_dict.get('sample_ID'), quant_id_dict.get(hierarchy_type)])
         #if there are multiple protein columns, merge them
        
        rows = df_subset.to_numpy()
        ions = []

        for row in rows: #iterate through dataframe
            ionlist = []
            count = 0
            ionstring = ""
            for lvl_idx in range(len(ion_hierarchy_local)):
                ionstring += f"{ion_hierarchy_local[lvl_idx]}"
                for sublvl in ion_headers_grouped[lvl_idx]:
                    ionstring+= f"_{row[count]}_"
                    count+=1
            ions.append(ionstring)
        df_subset['ion'] = ions
        df_subset = df_subset.reset_index()
        df_subset = df_subset.rename(columns = {quant_id_dict.get(hierarchy_type) : "quant_val"})

        ion_dfs.append(df_subset)
    input_df = pd.concat(ion_dfs,ignore_index=True)
    return input_df

def get_relvant_column_names(ion_dict, ion_hierarchy_local):
    ion_headers_merged = []
    ion_headers_grouped = []
    for lvl in ion_hierarchy_local:
        ion_headers_merged.extend(ion_dict.get(lvl)) 
        ion_headers_grouped.append(ion_dict.get(lvl))
    return ion_headers_merged, ion_headers_grouped

def split_extend_df(input_df, splitcol2sep, value_threshold=10):
    """reformats data that is stored in a condensed way in a single column. For example isotope1_intensity;isotope2_intensity etc. in Spectronaut

    Args:
        input_df ([type]): [description]
        splitcol2sep ([type]): [description]
        value_threshold([type]): [description]

    Returns:
        Pandas Dataframe: Pandas dataframe with the condensed items expanded to long format
    """
    if splitcol2sep==None:
        return input_df

    for split_col, separator in splitcol2sep.items():
        idx_name = f"{split_col}_idxs"
        split_col_series = input_df[split_col].str.split(separator)
        input_df = input_df.drop(columns = [split_col])

        input_df[idx_name] = [list(range(len(x))) for x in split_col_series]
        exploded_input = input_df.explode(idx_name)
        exploded_split_col_series = split_col_series.explode()

        exploded_input[split_col] = exploded_split_col_series #the column with the intensities has to come after to column with the idxs
        exploded_input = exploded_input.astype({split_col: float})
        exploded_input = exploded_input[exploded_input[split_col]>value_threshold]
        #exploded_input = exploded_input.rename(columns = {'var1': split_col})
    return exploded_input


In [ ]:
#export
def reformat_longtable_according_to_config_new(input_file, config_dict, sep = "\t",decimal = "."):
    """Reshape a long format proteomics results table (e.g. Spectronaut or DIA-NN) to a wide format table. 
    :param file input_file: long format proteomic results table
    :param string input_type: the configuration key stored in the config file (e.g. "diann_precursor")
    """
    relevant_cols = get_relevant_columns_config_dict(config_dict)
    input_df_it = pd.read_csv(input_file, sep = sep, decimal=decimal, usecols = relevant_cols, encoding ='latin1', chunksize=1000000)
    input_df_list = []
    for input_df_subset in input_df_it:
        input_df_subset = filter_input(config_dict.get("filters", {}), input_df_subset)
        if "ion_hierarchy" in config_dict.keys():
            input_df_subset = merge_protein_cols_and_ion_dict(input_df_subset, config_dict)
        else:
            input_df_subset = merge_protein_and_ion_cols(input_df_subset, config_dict)
        input_df_list.append(input_df_subset)
    
    
    input_df = pd.concat(input_df_list)
    
    input_df = input_df.astype({'quant_val': 'float'})
    input_reshaped = pd.pivot_table(input_df, index = ['protein', 'ion'], columns = config_dict.get("sample_ID"), values = 'quant_val', fill_value=0)
    if input_reshaped.iloc[:,0].replace(0, np.nan).median() <100: #when values are small, rescale by a constant factor to prevent rounding errors in the subsequent aq analyses
        input_reshaped = input_reshaped *10000
    
    input_reshaped = input_reshaped.reset_index()
    input_reshaped.to_csv(f"{input_file}.aq_reformat.tsv",  index = False,sep = "\t")
    
    return input_reshaped

In [ ]:
#export
def read_wideformat_table(peptides_tsv, config_dict):
    input_df = pd.read_csv(peptides_tsv,sep="\t", encoding ='latin1')
    filter_dict = config_dict.get("filters")
    protein_cols = config_dict.get("protein_cols")
    ion_cols = config_dict.get("ion_cols")
    input_df = filter_input(filter_dict, input_df)
    input_df = merge_protein_and_ion_cols(input_df, config_dict)
    if 'quant_prefix' in config_dict.keys():
        quant_prefix = config_dict.get('quant_prefix')
        headers = ['protein', 'ion'] + list(filter(lambda x: x.startswith(quant_prefix), input_df.columns))
        input_df = input_df[headers]
        input_df = input_df.rename(columns = lambda x : x.replace(quant_prefix, ""))

    input_df = input_df.reset_index()
    input_df.to_csv(f"{peptides_tsv}.aq_reformat.tsv",  index = False,sep = "\t")

    return input_df

In [ ]:
#export
def read_mq_peptides_table(peptides_tsv, pepheader = "Sequence", protheader = "Leading razor protein"):
    peps = pd.read_csv(peptides_tsv,sep="\t", encoding ='latin1')
    peps = peps[peps["Reverse"] != "+"]
    peps = peps[peps["Potential contaminant"] != "+"]
    if pepheader != None:
        peps = peps.rename(columns = {pepheader : "ion"})
    if protheader != None:
        peps = peps.rename(columns = {protheader: "protein"})
    headers = ['protein', 'ion'] + list(filter(lambda x: x.startswith("Intensity "), peps.columns))
    peps = peps[headers]
    peps = peps.rename(columns = lambda x : x.replace("Intensity ", ""))

    return peps

## check for already processed files

In [ ]:
#export
import os
def check_for_processed_runs_in_results_folder(results_folder):
    contained_condpairs = []
    folder_files = os.listdir(results_folder)
    result_files = list(filter(lambda x: "results.tsv" in x ,folder_files))
    for result_file in result_files:
        res_name = result_file.replace(".results.tsv", "")
        if ((f"{res_name}.normed.tsv" in folder_files) & (f"{res_name}.results.ions.tsv" in folder_files)):
            contained_condpairs.append(res_name)
    return contained_condpairs

## Wrapper functions

In [ ]:
#export
import pandas as pd
import os
import pkg_resources
import pathlib

def import_data(input_file,  verbose=True):
    """
    Function to import peptide level data. Depending on available columns in the provided file,
    the function identifies the type of input used (e.g. Spectronaut, MaxQuant, DIA-NN), reformats if necessary
    and returns a generic wide-format dataframe
    :param file input_file: quantified peptide/ion -level data
    :param file results_folder: the folder where the AlphaQuant outputs are stored
    """
    config_file = os.path.join(pathlib.Path(__file__).parent.absolute(), "..", "intable_config.yaml") #the yaml config is located one directory below the python library files
    config_dict = load_config(config_file)
    type2relevant_columns = get_type2relevant_cols(config_dict)

    
    filename = str(input_file)
    if '.csv' in filename:
        sep=','
    if '.tsv' in filename:
        sep='\t'
    if '.txt' in filename:
        sep='\t'

    if 'sep' not in locals():
        raise TypeError(f"neither of the file extensions (.tsv, .csv, .txt) detected for file {input_file}! Your filename has to contain one of these extensions. Please modify your file name accordingly.")
    
    if "aq_reformat" in input_file:
        data = pd.read_csv(input_file, sep = "\t", encoding ='latin1')
        return data

    uploaded_data_columns = set(pd.read_csv(input_file, sep=sep, nrows=1, encoding ='latin1').columns)
    
    for input_type in type2relevant_columns.keys():

        relevant_columns = type2relevant_columns.get(input_type)
        relevant_columns = [x for x in relevant_columns if x] #filter None values
        print(f"recols\t {relevant_columns}")
        if set(relevant_columns).issubset(uploaded_data_columns):
            config_dict_type =  config_dict.get(input_type)
            format = config_dict_type.get("format")
            if verbose:
                print(f"{input_type} headers in format {format} detected. Importing and re-formating.")
            if format == "longtable":
                data = reformat_longtable_according_to_config_new(input_file, config_dict_type, sep = sep)
            elif format == "widetable":
                data = read_wideformat_table(input_file, config_dict_type)
            else:
                raise Exception("format: not specified in intable_config.yaml")
            return data

    #if non of the cases match, return error
    raise TypeError(f'Input data format for {input_file} not known.')


In [ ]:
#export
import pandas as pd

def get_samplenames(data):
    """extracts the names of the samples of the AQ input dataframe"""
    names = list(data.columns)
    names.remove('protein')
    names.remove('ion')
    return names


In [ ]:
#export

import pandas as pd

def load_samplemap(samplemap_file):
    file_ext = os.path.splitext(samplemap_file)[-1]
    if file_ext=='.csv':
        sep=','
    if (file_ext=='.tsv') | (file_ext=='.txt'):
        sep='\t'
    
    if 'sep' not in locals():
        raise TypeError(f"neither of the file extensions (.tsv, .csv, .txt) detected for file {samplemap_file}! Your filename has to end with one of these extensions. Please modify your file name accordingly.")
        sep = "\t"

    return pd.read_csv(samplemap_file, sep = sep, encoding ='latin1')

In [ ]:
#export
def prepare_loaded_tables(data_df, samplemap_df):
    """
    Integrates information from the peptide/ion data and the samplemap, selects the relevant columns and log2 transforms intensities.
    """
    samplemap_df = samplemap_df[samplemap_df["condition"]!=""] #remove rows that have no condition entry
    filtvec_not_in_data = [(x in data_df.columns) for x in samplemap_df["sample"]] #remove samples that are not in the dataframe
    samplemap_df = samplemap_df[filtvec_not_in_data]
    headers = ['protein'] + samplemap_df["sample"].to_list()
    data_df = data_df.set_index("ion")
    for sample in samplemap_df["sample"]:
        data_df[sample] = np.log2(data_df[sample].replace(0, np.nan))
    return data_df[headers], samplemap_df